# Project 1

In [ ]:
#Importing pandas as pd
import pandas as pd

#Setting options to display full data frame rows
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
#Reading in bikeDetails data set using pandas
bike_data = pd.read_csv("bikeDetails.csv")

In [ ]:
#Displays the first 5 observations from the bike_data data set
bike_data.head()

,name,selling_price,year,seller_type,owner,km_driven,ex_showroom_price
1056,Activa 3g,17000,2010,Individual,1st owner,500000,52000.0
1057,Honda CB twister,16000,2012,Individual,1st owner,33000,51000.0
1058,Bajaj Discover 125,15000,2013,Individual,2nd owner,35000,57000.0
1059,Honda CB Shine,12000,2009,Individual,1st owner,53000,58000.0
1060,Bajaj Pulsar 150,10000,2008,Individual,1st owner,92233,75000.0


## Ashley's EDA

## George's EDA

In [ ]:
#using the .describe() method to get a first look at the numeric variables in the dataset. Although you would not necessarily perform math on the listed years for each bike, seeing info like the average year is meaningful.
bike_data.describe()

,selling_price,year,km_driven,ex_showroom_price
count,1061.000000,1061.000000,1061.000000,6.260000e+02
mean,59638.151744,2013.867107,34359.833176,8.795871e+04
std,56304.291973,4.301191,51623.152702,7.749659e+04
min,5000.000000,1988.000000,350.000000,3.049000e+04
25%,28000.000000,2011.000000,13500.000000,5.485200e+04
50%,45000.000000,2015.000000,25000.000000,7.275250e+04
75%,70000.000000,2017.000000,43000.000000,8.703150e+04
max,760000.000000,2020.000000,880000.000000,1.278000e+06


In [ ]:
bike_data.columns
#note: our columns are name, selling_price, year, seller_type, owner, km_driven, and ex_showroom_price

Index(['name', 'selling_price', 'year', 'seller_type', 'owner', 'km_driven',
       'ex_showroom_price'],
      dtype='object')

In [66]:
#bike_data_numeric = bike_data['selling_price', 'km_driven', 'ex_showrooom_price']

#previously I tried to do what I was doing above exactly in the way he does it
#in the video and got the same error

#Fixed to grab desired columns from bike_data for all rows
bike_data_numeric = bike_data.loc[:, ['selling_price', 'km_driven', 'ex_showroom_price']]

bike_data_numeric


,selling_price,km_driven,ex_showroom_price
0,175000,350,NaN
1,45000,5650,NaN
2,150000,12000,148114.0
3,65000,23000,89643.0
4,20000,21000,NaN
5,18000,60000,53857.0
6,78500,17000,87719.0
7,180000,39000,NaN
8,30000,32000,NaN
9,50000,42000,60122.0


In [ ]:
#finding mean and median of all the numeric variables using the .mean and .median methods
bike_data['selling_price'].mean()

59638.151743638075

In [ ]:
bike_data['selling_price'].median()

45000.0

In [ ]:
bike_data['km_driven'].mean()

34359.83317624882

In [ ]:
bike_data['km_driven'].median()

25000.0

In [ ]:
bike_data['ex_showroom_price'].mean()

87958.71405750798

In [ ]:
bike_data['ex_showroom_price'].median()

72752.5

In [ ]:
#similarly finding standard deviations of the numeric variables
bike_data.selling_price.std()

56304.29197302419

In [ ]:
bike_data.km_driven.std()

51623.15270159632

In [ ]:
bike_data.ex_showroom_price.std()

77496.5871894531

In [ ]:
#similarly finding more fine-tuned quantiles
bike_data.selling_price.quantile(q = [.05, .10, .15, .20, .25, .30, .35, .40, .45, .50, .55, .60, .65, .70, .75, .80, .85, .90, .95])

0.05     15000.0
0.10     20000.0
0.15     23000.0
0.20     25000.0
0.25     28000.0
0.30     30000.0
0.35     35000.0
0.40     38000.0
0.45     40000.0
0.50     45000.0
0.55     47000.0
0.60     50000.0
0.65     58000.0
0.70     62000.0
0.75     70000.0
0.80     80000.0
0.85    100000.0
0.90    120000.0
0.95    150000.0
Name: selling_price, dtype: float64

In [1]:
bike_data.km_driven.quantile(q = [.05, .10, .15, .20, .25, .30, .35, .40, .45, .50, .55, .60, .65, .70, .75, .80, .85, .90, .95])

0.05     3500.0
0.10     6000.0
0.15     8000.0
0.20    10950.0
0.25    13500.0
0.30    15000.0
0.35    17500.0
0.40    20000.0
0.45    22000.0
0.50    25000.0
0.55    28000.0
0.60    31000.0
0.65    35000.0
0.70    39448.0
0.75    43000.0
0.80    49000.0
0.85    52500.0
0.90    65000.0
0.95    80000.0
Name: km_driven, dtype: float64

In [ ]:
bike_data.ex_showroom_price.quantile(q = [.05, .10, .15, .20, .25, .30, .35, .40, .45, .50, .55, .60, .65, .70, .75, .80, .85, .90, .95])

0.05     47277.00
0.10     50267.00
0.15     52203.25
0.20     54299.00
0.25     54852.00
0.30     57549.00
0.35     59362.25
0.40     61653.00
0.45     65669.50
0.50     72752.50
0.55     75936.00
0.60     78712.00
0.65     81508.00
0.70     84751.00
0.75     87031.50
0.80     94000.00
0.85    117926.00
0.90    147000.00
0.95    181445.00
Name: ex_showroom_price, dtype: float64

In [ ]:
#interquartile range for selling_price
q1 = bike_data.selling_price.quantile(q = [.25])
q3 = bike_data.selling_price.quantile(q = [.75])
q1
q3
q3[.75] - q1[.25]

42000.0

In [ ]:
#finding correlation between the numeric variables in the data set
bike_data.corr()

,selling_price,year,km_driven,ex_showroom_price
selling_price,1.000000,0.402188,-0.212937,0.918583
year,0.402188,1.000000,-0.288675,0.126378
km_driven,-0.212937,-0.288675,1.000000,-0.122011
ex_showroom_price,0.918583,0.126378,-0.122011,1.000000


We see that selling_price and ex_showroom_price are highly correlated, in fact even more highly correlated than I anticipated or what it looked like from a cursory look at the data. They in fact boast a roughly 92% correlation with one another. Also, selling_price and km_driven are negatively correlated as one would expect (the more use a vehicle has the lower the price all else being equal). Insofar as it is meaningful the positive correlation shown between year and selling_price also makes sense because a larger number for year represents a newer vehicle. One might expect that until a bike is so old as to be considered a collector's item or an antique that an older bike would be worth less.

In [ ]:
bike_data.groupby("seller_type").mean()

,selling_price,year,km_driven,ex_showroom_price
seller_type,,,,
Dealer,46666.666667,2011.000000,35258.833333,88536.000000
Individual,59711.923223,2013.883412,34354.720379,87955.934189


In [ ]:
bike_data.groupby("owner").mean()

,selling_price,year,km_driven,ex_showroom_price
owner,,,,
1st owner,60022.813853,2014.200216,32816.583333,84362.796763
2nd owner,55666.658537,2012.056911,39288.991870,112554.651515
3rd owner,55727.272727,2007.818182,33292.181818,64611.000000
4th owner,118333.333333,2007.666667,311500.000000,534000.000000


We can see that dealers tend to sell their bikes for about 13,000 dollars less than individuals do as well as sell slightly older bikes that have been ridden a little farther. Very interestingly, in the second set of grouped means we see that by far the most lucrative sales occur by the 4th owner. Intuitively one might expect a bike that has been sold at least three times before might be worth less, but that is clearly not the case. This owner status, particularly in the case of "4th owner" could be a powerful predictive attribute.